In [ ]:
import os
from math import ceil
import tensorflow as tf
from keras import models, layers
from keras.applications import ResNet50
from keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ROOT = "Oral Cancer Prediction"
DATA_PATH = os.path.join(ROOT, "assets", "dataset")

IMG_SIZE = (224, 224)

BATCH_SIZE = 32

In [ ]:
ds = image_dataset_from_directory(
    DATA_PATH, image_size=IMG_SIZE, batch_size=BATCH_SIZE, shuffle=True, seed=123
)

In [ ]:
CLASS_NAMES = ds.class_names
print(CLASS_NAMES)

class_counts = {
    0: os.listdir(os.path.join(DATA_PATH, CLASS_NAMES[0])).__len__(),
    1: os.listdir(os.path.join(DATA_PATH, CLASS_NAMES[1])).__len__(),
}
print(class_counts)

In [ ]:
for img_batch, label_batch in ds.take(1):
    print(img_batch.shape)
    print(img_batch[0].shape)

Batches of 32 images of size (224, 224) in 3 color channels.


## Split dataset


In [ ]:
train_split = 0.8
val_split = 0.1
train_size = int(len(ds) * train_split)
val_size = ceil(len(ds) * val_split)
print(f"Number of batches:   {len(ds)}")
print(f"Training batches:    {train_size}")
print(f"Validating batches:  {val_size}")
print(f"Testing batches:     {len(ds) - train_size - val_size}")

In [ ]:
train_ds = ds.take(train_size)
test_ds = ds.skip(train_size)
val_ds = test_ds.take(val_size)
test_ds = test_ds.skip(val_size)

Creating a function to do all these things with shuffling


In [ ]:
def dataset_partitions(
    ds, train_split=0.8, val_split=0.1, shuffle=True, shuffle_size=1000
):
    if shuffle:
        ds.shuffle(shuffle_size)

    ds_size = int(ds.reduce(0, lambda x, _: x + 1).numpy())
    train_size = int(ds_size * train_split)
    val_size = ceil(ds_size * val_split)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = dataset_partitions(ds)

print(f"Number of batches  : {len(ds)}")
print(f"Training batches   : {len(train_ds)}")
print(f"Validating batches : {len(val_ds)}")
print(f"Testing batches    : {len(test_ds)}")

In [ ]:
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Bulding a model


In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomFlip("vertical"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
)

## Manual CNN model


In [ ]:
input_shape = (BATCH_SIZE, *IMG_SIZE, 3)
n_classes = 2

model = models.Sequential(
    [
        layers.Resizing(*IMG_SIZE),
        layers.Rescaling(1.0 / 255),
        data_augmentation,
        layers.Conv2D(
            32, kernel_size=(3, 3), activation="relu", input_shape=input_shape
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [ ]:
# Compile
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC()],
)

# Train
history = model.fit(train_ds, validation_data=val_ds, batch_size=BATCH_SIZE, epochs=10)

In [ ]:
# Get True and Predicted results
y_true = np.concatenate([y.numpy() for x, y in val_ds], axis=0)
y_pred_probs = model.predict(val_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# Generate Classification report
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))

## Transfer Learning using ResNet50


In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False)
base_model.trainable = False

In [ ]:
inputs = layers.Input(shape=(*IMG_SIZE, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1.0 / 255)(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = models.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# Compile
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC()],
)

# Train
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

In [ ]:
# Get True and Predicted results
y_true = np.concatenate([y.numpy() for x, y in val_ds], axis=0)
y_pred_probs = model.predict(val_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# Generate Classification report
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))

## Dealing with Imbalanced datset

Methods:-

1. using custom weights
2. undersampling larger dataset
3. duplicate oversampling smaller dataset
4. SMOTE oversampling smaller dataset (mod = imblearn)
5. ensemble technique
6. focal loss


### M1. Undersampling


In [ ]:
min_class_count = min(class_counts.values())

under_ds = (
    ds.unbatch()
    .filter(lambda x, y: y == 0)
    .take(min_class_count)
    .concatenate(ds.unbatch().filter(lambda x, y: y == 1).take(min_class_count))
    .batch(BATCH_SIZE)
)

In [ ]:
print("No. of batches:", end=" ")
int(under_ds.reduce(0, lambda x, _: x + 1).numpy())

In [ ]:
from collections import Counter

counter = Counter()
for _, labels in under_ds.unbatch():
    counter[int(labels.numpy())] += 1

print("Balanced class counts:", counter)

In [ ]:
train_under_ds, val_under_ds, test_under_ds = dataset_partitions(under_ds)

train_under_ds = train_under_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_under_ds = val_under_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_under_ds = test_under_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False)
base_model.trainable = False

inputs = layers.Input(shape=(*IMG_SIZE, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1.0 / 255)(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_under = models.Model(inputs, outputs)

model_under.summary()

In [ ]:
# Compile
model_under.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC()],
)

# Train
history = model_under.fit(train_under_ds, validation_data=val_under_ds, epochs=10)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 257s 32s/step - accuracy: 0.8303 - auc_2: 0.0551 - loss: 0.9696 - val_accuracy: 0.0000e+00 - val_auc_2: 0.0000e+00 - val_loss: 3.4568
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 188s 27s/step - accuracy: 0.8922 - auc_2: 0.4803 - loss: 0.2242 - val_accuracy: 1.0000 - val_auc_2: 0.0000e+00 - val_loss: 0.2799
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 144s 21s/step - accuracy: 0.1357 - auc_2: 0.0328 - loss: 1.3943 - val_accuracy: 1.0000 - val_auc_2: 0.0000e+00 - val_loss: 0.5957
Epoch 4/10
5/7 ━━━━━━━━━━━━━━━━━━━━ 54s 27s/step - accuracy: 0.5320 - auc_2: 0.0185 - loss: 0.7037     

In [ ]:
# Get True and Predicted results
y_true = np.concatenate([y.numpy() for x, y in val_under_ds], axis=0)
y_pred_probs = model.predict(val_under_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# Generate Classification report
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))